In [49]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 경로 설정
base_dir = "C:/Users/Admin/Desktop/data"
categories = ["back", "squat", "side"]

def load_data(base_dir, categories, seq_len=120):
    """
    데이터 로드 및 크기 조정 (패딩 또는 잘라내기).
    Args:
        base_dir (str): 데이터가 저장된 기본 경로.
        categories (list): 행동 카테고리 리스트.
        seq_len (int): 고정된 시퀀스 길이.
    Returns:
        np.array: 데이터 배열.
        np.array: 라벨 배열.
    """
    data = []
    labels = []
    label_map = {category: idx for idx, category in enumerate(categories)}  # 라벨 매핑

    for category in categories:
        category_path = os.path.join(base_dir, category, "좌표")
        print(f"Checking files in: {category_path}")
        for i in range(1, 101):  # keypoints_1 ~ keypoints_100
            file_path = os.path.join(category_path, f"keypoints_{i}.npy")
            if os.path.exists(file_path):
                keypoints = np.load(file_path)  # 데이터 로드
                if keypoints.shape[0] > seq_len:  # 길이가 seq_len보다 길면 잘라내기
                    keypoints = keypoints[:seq_len]
                elif keypoints.shape[0] < seq_len:  # 길이가 seq_len보다 짧으면 패딩
                    pad_width = seq_len - keypoints.shape[0]
                    keypoints = np.pad(keypoints, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                data.append(keypoints)
                labels.append(label_map[category])
            else:
                print(f"File not found: {file_path}")

    return np.array(data), np.array(labels)

# 수정된 데이터 로드 실행
data, labels = load_data(base_dir, categories)
print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Checking files in: C:/Users/Admin/Desktop/data\back\좌표
Checking files in: C:/Users/Admin/Desktop/data\squat\좌표
Checking files in: C:/Users/Admin/Desktop/data\side\좌표
Data shape: (300, 120, 33, 3), Labels shape: (300,)


In [50]:
import os
import numpy as np

# 데이터 경로 및 라벨 매핑
data_paths = {
    "back": "C:/Users/Admin/Desktop/data/back/좌표",
    "squat": "C:/Users/Admin/Desktop/data/squat/좌표",
    "side": "C:/Users/Admin/Desktop/data/side/좌표"
}
labels_map = {"back": 0, "squat": 1, "side": 2}  # 라벨 매핑

def load_data(data_paths, labels_map, seq_len=120):
    """
    데이터 로드 및 크기 조정 (패딩 또는 잘라내기).
    Args:
        data_paths (dict): 카테고리별 데이터 경로.
        labels_map (dict): 카테고리와 라벨 매핑.
        seq_len (int): 고정된 시퀀스 길이.
    Returns:
        np.array: 데이터 배열.
        np.array: 라벨 배열.
    """
    data = []
    labels = []
    for category, path in data_paths.items():
        print(f"Checking files in: {path}")
        for i in range(1, 101):  # keypoints_1 ~ keypoints_100
            file_path = os.path.join(path, f"keypoints_{i}.npy")
            if os.path.exists(file_path):
                keypoints = np.load(file_path)  # 데이터 로드
                if keypoints.shape[0] > seq_len:  # 길이가 seq_len보다 길면 잘라내기
                    keypoints = keypoints[:seq_len]
                elif keypoints.shape[0] < seq_len:  # 길이가 seq_len보다 짧으면 패딩
                    pad_width = seq_len - keypoints.shape[0]
                    keypoints = np.pad(keypoints, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                data.append(keypoints)
                labels.append(labels_map[category])
            else:
                print(f"File not found: {file_path}")
    return np.array(data), np.array(labels)

In [51]:
import os

for label_name, path in data_paths.items():
    print(f"Checking files in: {path}")
    print(os.listdir(path))

# 데이터 로드
X, y = load_data(data_paths, labels_map)

# 데이터 크기 확인
print("데이터 크기:", X.shape, y.shape)  # Expected output: (30, frames, joints, coords), (30,)

Checking files in: C:/Users/Admin/Desktop/data/back/좌표
['keypoints_1.npy', 'keypoints_10.npy', 'keypoints_100.npy', 'keypoints_11.npy', 'keypoints_12.npy', 'keypoints_13.npy', 'keypoints_14.npy', 'keypoints_15.npy', 'keypoints_16.npy', 'keypoints_17.npy', 'keypoints_18.npy', 'keypoints_19.npy', 'keypoints_2.npy', 'keypoints_20.npy', 'keypoints_21.npy', 'keypoints_22.npy', 'keypoints_23.npy', 'keypoints_24.npy', 'keypoints_25.npy', 'keypoints_26.npy', 'keypoints_27.npy', 'keypoints_28.npy', 'keypoints_29.npy', 'keypoints_3.npy', 'keypoints_30.npy', 'keypoints_31.npy', 'keypoints_32.npy', 'keypoints_33.npy', 'keypoints_34.npy', 'keypoints_35.npy', 'keypoints_36.npy', 'keypoints_37.npy', 'keypoints_38.npy', 'keypoints_39.npy', 'keypoints_4.npy', 'keypoints_40.npy', 'keypoints_41.npy', 'keypoints_42.npy', 'keypoints_43.npy', 'keypoints_44.npy', 'keypoints_45.npy', 'keypoints_46.npy', 'keypoints_47.npy', 'keypoints_48.npy', 'keypoints_49.npy', 'keypoints_5.npy', 'keypoints_50.npy', 'keypoin

In [52]:
def standardize_data(data):
    """
    데이터 표준화 (평균 0, 표준편차 1).
    Args:
        data (np.array): 원본 데이터, Shape: (samples, frames, joints, coords).
    Returns:
        np.array: 표준화된 데이터.
    """
    mean = data.mean(axis=(1, 2, 3), keepdims=True)  # 샘플별 평균 계산
    std = data.std(axis=(1, 2, 3), keepdims=True)  # 샘플별 표준편차 계산
    standardized_data = (data - mean) / (std + 1e-8)  # 표준화
    return standardized_data

# 데이터 표준화 적용
X_standardized = standardize_data(X)

In [53]:
# 데이터 분할 (훈련/검증/테스트 세트)
X_train, X_temp, y_train, y_temp = train_test_split(X_standardized, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}, Test shape: {X_test.shape}")

Train shape: (210, 120, 33, 3), Validation shape: (45, 120, 33, 3), Test shape: (45, 120, 33, 3)


In [54]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)  # LSTM 출력
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력만 사용
        return out
    
# 모델 초기화
input_dim = X_train.shape[2] * X_train.shape[3]  # 관절 수 * 좌표 수
hidden_dim = 256
output_dim = len(labels_map)  # 클래스 수 (등, 스쿼트, 옆구리)
model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# 데이터 로더 생성
train_dataset = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
val_dataset = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

In [55]:
from sklearn.utils.class_weight import compute_class_weight

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 손실 함수 가중치 조정
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float))

In [56]:
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torch

# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 학습률 스케줄러 추가
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

# EarlyStopping 객체 생성
early_stopping = EarlyStopping(patience=7, min_delta=0.0001)

# 학습 루프
num_epochs = 100  # 에포크를 100번으로 설정
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        X_batch = X_batch.view(X_batch.size(0), -1, input_dim)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 검증 단계
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.view(X_batch.size(0), -1, input_dim)
            y_pred = model(X_batch)
            val_loss += criterion(y_pred, y_batch).item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Validation Loss: {val_loss / len(val_loader):.4f}")

    # Early Stopping 확인
    early_stopping(val_loss / len(val_loader))
    if early_stopping.early_stop:
        print("Early stopping triggered. Training stopped.")
        break


Epoch 1/100, Train Loss: 0.8206, Validation Loss: 0.8040
Epoch 2/100, Train Loss: 0.5811, Validation Loss: 0.5091
Epoch 3/100, Train Loss: 0.5322, Validation Loss: 0.4603
Epoch 4/100, Train Loss: 0.5053, Validation Loss: 0.5846
Epoch 5/100, Train Loss: 0.5348, Validation Loss: 0.4727
Epoch 6/100, Train Loss: 0.4396, Validation Loss: 0.2168
Epoch 7/100, Train Loss: 0.2819, Validation Loss: 0.5482
Epoch 8/100, Train Loss: 0.4286, Validation Loss: 0.1581
Epoch 9/100, Train Loss: 0.2669, Validation Loss: 0.1172
Epoch 10/100, Train Loss: 0.1880, Validation Loss: 0.1299
Epoch 11/100, Train Loss: 0.1404, Validation Loss: 0.1032
Epoch 12/100, Train Loss: 0.1762, Validation Loss: 0.1252
Epoch 13/100, Train Loss: 0.1420, Validation Loss: 0.0468
Epoch 14/100, Train Loss: 0.0904, Validation Loss: 0.0403
Epoch 15/100, Train Loss: 0.0531, Validation Loss: 0.0167
Epoch 16/100, Train Loss: 0.0492, Validation Loss: 0.1002
Epoch 17/100, Train Loss: 0.0493, Validation Loss: 0.0303
Epoch 18/100, Train Los

In [57]:
print(f"Data min: {X.min()}, Data max: {X.max()}")
print(f"Class distribution: {np.bincount(y)}")

Data min: -1.3231176137924194, Data max: 1.0206526517868042
Class distribution: [100 100 100]


In [58]:
# 테스트 데이터 평가
from sklearn.metrics import classification_report
model.eval()
y_true, y_pred = [], []
test_loader = DataLoader(TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).long()), batch_size=4)
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.view(X_batch.size(0), -1, input_dim)  # 차원 변환
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

print(classification_report(y_true, y_pred, target_names=list(labels_map.keys())))

              precision    recall  f1-score   support

        back       1.00      1.00      1.00        15
       squat       1.00      1.00      1.00        15
        side       1.00      1.00      1.00        15

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [59]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred))

[[15  0  0]
 [ 0 15  0]
 [ 0  0 15]]


In [60]:
import torch
import torch.nn as nn
import os
'''
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)  # LSTM 출력
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력만 사용
        return out

# 모델 초기화
input_dim = 34  # 입력 크기: 34
hidden_dim = 64  # LSTM의 은닉 상태 크기
output_dim = 3  # 출력 클래스 수 (e.g., Back, Squat, Side)
num_layers = 3  # LSTM 레이어 수
model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers)
'''

# TorchScript로 변환
scripted_model = torch.jit.script(model)

# 경로 설정
output_dir = "C:/Users/Admin/Desktop"
os.makedirs(output_dir, exist_ok=True)  # 디렉토리가 없으면 생성
model_path = os.path.join(output_dir, "lstm_model_scripted.pt")

# TorchScript 모델 저장
scripted_model.save(model_path)
print(f"Scripted model saved to {model_path}")


Scripted model saved to C:/Users/Admin/Desktop\lstm_model_scripted.pt
